# Price calculator

This file shows an example of how to calculate the price of openai API calls.

In [18]:
from utils import get_openai_api_key, get_openai_base_url
import openai
from pprint import pprint
import json
from typing import List, Dict, Optional, Any, Union

In [19]:
def chat_demo(model: str, messages: List[Dict[str, Any]]) -> Dict[str, Any]:
    api_keys = get_openai_api_key()
    base_url = get_openai_base_url()

    client = openai.OpenAI(api_key=api_keys, base_url=base_url)
    response = client.chat.completions.create(model=model, messages=messages)
    
    return response

In [ ]:

model = "gpt-3.5-turbo"
message = [
    {
        "role": "user",
        "content": "Translate the following English text to French: 'Hello, how are you?'",
    }
]

response = chat_demo(model=model, messages=message)
response_list = []
response_list.append(response)

print(f"Response:\n{response}")
pprint(response.to_dict())

In [ ]:
# load price data: $/1k tokens
with open('price_usd_per_1k_tokens.json') as f:
    price = json.load(f)

print(type(price))

In [ ]:

def calculate_response_costs(price_list: dict, response: openai.ChatCompletion):
    """
    Calculate the cost of a single response from OpenAI's API.
    Note: The costs could be all zeros if the number of tokens is very low.
    """
    price = price_list    

    model_name = response.model
    usage = response.usage.to_dict()

    model_price = price.get(model_name, {})
    input_cost = (
        model_price.get("input_tokens", 0) * usage.get("prompt_tokens", 0) / 1000
    )
    output_cost = (
        model_price.get("output_tokens", 0) * usage.get("completion_tokens", 0) / 1000
    )
    total_cost = input_cost + output_cost

    # print(f"The total cost for using {model_name} is ${total_cost:.5f}")
    return input_cost, output_cost, total_cost

calculate_response_costs(price, response)

In [ ]:
def calculate_total_tokens(responses: list) -> Dict[str, Dict[str, int]]:
    """
    Calculate the total tokens used for each model in the responses.
    """
    total_tokens = {}

    for response in responses:
        model_name = response.model
        usage = response.usage.to_dict()

        if model_name not in total_tokens:
            total_tokens[model_name] = {"input_tokens": 0, "output_tokens": 0}

        total_tokens[model_name]["input_tokens"] += usage.get("prompt_tokens", 0)
        total_tokens[model_name]["output_tokens"] += usage.get("completion_tokens", 0)
        total_tokens[model_name]["total_tokens"] = (
            total_tokens[model_name]["input_tokens"] + total_tokens[model_name]["output_tokens"]
        )

    return total_tokens

calculate_total_tokens(response_list)

In [ ]:
def calculate_total_costs(price_list: dict, responses: list) -> Dict[str, float]:
    """
    Calculate the total costs for each model in the responses.
    """
    total_tokens = calculate_total_tokens(responses)
    total_costs = {}

    for model_name, tokens in total_tokens.items():
        model_price = price_list.get(model_name, {})
        input_cost = model_price.get("input_tokens", 0) * tokens["input_tokens"] / 1000
        output_cost = model_price.get("output_tokens", 0) * tokens["output_tokens"] / 1000
        total_costs[model_name] = input_cost + output_cost

    return total_costs

calculate_total_costs(price, response_list)